## Evaluation and Calibration of the Empirical Models in TELL

This notebook is meant to be a landing page for exploring the empirical models that underpin `tell`. It will set up the meteorological variables we use and analyze different aspects of the model development and evaluation.

In [ ]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell


In [ ]:
# Identify the current working directory, the "tell_data" directory where the data will is stored, and the image output directory:
current_dir =  os.path.join(os.path.dirname(os.getcwd()))
tell_data_dir = os.path.join(current_dir, r'tell_data')
tell_image_dir = os.path.join(tell_data_dir, r'outputs', r'visualizations', r'mlp_evaluation')

# If the "tell_image_dir" subdirectory doesn't exist then create it:
if not os.path.exists(tell_image_dir):
   os.makedirs(tell_image_dir)

### Variable Correlations

The first thing we'll look at is the relationship between the predictive variables in `tell` and the historical electricity demand for different Balancing Authorities (BAs). The function below plots the correlation between the different variables in the historical observed data that underpins `tell`. In general, demand is fairly well correlated with most of the meteorological variables, with the strongest dependency being on 2-m air temperature (T2). The stength of the correlations varies significantly from BA-to-BA. Demand in some BAs is closely tied to variations in weather (e.g., NEVP, ERCO, FPL). In more moderate climates (e.g., SCL, BPAT) the relationships are weaker. This pattern is useful for interpreting the performance of the empircal models in `tell`.

In [ ]:
# Plot the correlation between predictive variables in TELL and the historical demand in a given BA. Note that if you set the 'ba_to_plot' variable to "All" the function 
# will compute and plot the average correlation across all BAs:
tell.plot_ba_variable_correlations(ba_to_plot = 'NEVP', 
                                   data_input_dir = tell_data_dir,
                                   image_output_dir = tell_image_dir,
                                   image_resolution = 150,
                                   save_images = True)


### Empirical Model Training

Next we'll train the `tell` models using the standard settings found in the `mlp_settings.yml` file. By default, the `tell` multi-layer perceptron (MLP) models are trained on historical weather and load data from 2016-2018 and evaluated using data from 2019. In addition to the MLP models, `tell` trains a linear model for each BA that uses the hourly evolution of total population within the BA's service territory to predict the residuals from the MLP models. The quantitative effect of these linar models is evaluated later in the notebook.

In [ ]:
# Train the empirical models using the standard settings:
prediction_df, validation_df = tell.train_batch(target_region_list = (tell.get_balancing_authority_to_model_dict().keys()),
                                                data_dir = os.path.join(tell_data_dir, r'outputs', r'compiled_historical_data'),
                                                n_jobs = -1)
        
# View the validation dataframe that contains error statistics for the trained models:
validation_df


### Empirical Model Evaluation

After training the models, we'll visualize their performance by plotting the error characteristics for each of the BAs in `tell`. In general, the default training settings results in models that perform quite well. 76% (41/54) of the BAs have an R2 value greater than 0.75 while 89% (48/54) have a MAPE under 10%.

In [ ]:
# Plot the statistical performance (e.g., RMS_ABS, RMS_NORM, MAPE, or R2) of the predictive models across all the BAs in TELL:
tell.plot_mlp_summary_statistics(validation_df, 
                                 image_output_dir = tell_image_dir,
                                 image_resolution = 150,
                                 save_images = True)


It's illustrative to look at the error metrics as a function of load. To do this, we calculate the mean hourly load for each BA during the evaluation year and then plot the error statistics as a function of that mean load. Analyzing the data in this way demonstrates that the BAs with the poorly performing emprical models are almost universally the smaller BAs. The largest BAs, which are critically important for the overall demand on the grid, generally perform quite well. Of the 10 BAs with the largest mean demand, 9/10 have a MAPE value under 5% and an R2 value greater than 0.85. Conversely, of the 10 worst performing BAs (judged by their MAPE value), 7/10 have an average hourly load less than 1200 MWh.

In [ ]:
# Plot the statistical performance (e.g., RMS_ABS, RMS_NORM, MAPE, or R2) of the predictive models as a function of mean load for each BA and update the validation dataframe:
validation_df = tell.plot_mlp_errors_vs_load(prediction_df, 
                                             validation_df, 
                                             image_output_dir = tell_image_dir,
                                             image_resolution = 150,
                                             save_images = True)


In [ ]:
validation_df.sort_values(by='Mean_Load_MWh', ascending=False).head(10)

In [ ]:
validation_df.sort_values(by='MAPE', ascending=False).head(10)

The next set of plots analyzes the predicted and observed demands for individual BAs in order to shed more light on the performance of the `tell` empirical models. The first function plots the time-series and histograms of observed and predicted demands for a given BA. Using this function you can see the strong performance of `tell` in larger BAs (e.g., PJM, MISO, ERCO). They also shed some clues into the poor performance in smaller BAs. For example, the data from SEC shows bad observations that weren't caught by the data cleanining functions that are ran before the MLP models are trained. Despite its poor performance, the small loads present in SEC allow us to retain the model for that BA even though it is poor without impacting the overall fidelity of `tell`.

In [ ]:
tell.plot_mlp_ba_time_series(prediction_df, 
                             ba_to_plot = 'PJM',
                             image_output_dir = tell_image_dir,
                             image_resolution = 150,
                             save_images = True)


Next let's take a look at the performance of the `tell` empirical models during the peak week of demand during the evaluation period. To find the peak demand week we compute a weekly rolling mean value of predicted demand using exponentially-weighted windows and then subset the data to +/- 3.5 days around the peak value. This analysis can be used to understand how well `tell` is capturing peak loads.

In [ ]:
tell.plot_mlp_ba_peak_week(prediction_df, 
                           ba_to_plot = 'PJM',
                           image_output_dir = tell_image_dir,
                           image_resolution = 150,
                           save_images = True)


### Impact of the Supplemental Linear Model

The final thing we'll explore is the impact of training a linear model on the residuals of the MLP models in `tell`. This was done because population within the BAs evolves on an annual scale as opposed to hourly like the meteorological variables and thus population is rarely selected as a strong predictor in the MLP models. Nonetheless, we know that the gradual evolution of population within a BA can impact the evolution of demand. In this final section of the notebooks we retrain the `tell` empirical models without invoking the linear model adjustment step (i.e., `mlp_linear_adjustment = False`). In most BAs the models with the linear adjustment perform slightly better than those without. This shows up most clearly in the MAPE value plotted below.

In [ ]:
prediction_df_nolinear, validation_df_nolinear = tell.train_batch(target_region_list = (tell.get_balancing_authority_to_model_dict().keys()),
                                                                  mlp_linear_adjustment = False,
                                                                  data_dir = os.path.join(tell_data_dir, r'outputs', r'compiled_historical_data'),
                                                                  n_jobs = -1)

validation_df_nolinear

In [ ]:
# Plot the R2 and MAPE score for each BA using the validation_df for the models with and without the linear adjustment step:
impact_df = tell.plot_mlp_linear_model_impact(validation_df, 
                                              validation_df_nolinear,
                                              image_output_dir = tell_image_dir,
                                              image_resolution = 150,
                                              save_images = True)
